# 11. 심층신경망 훈련

1. Vanishing Gradient(그래디언트 소실) or Exploding Gradient(그래디언트 폭주) 문제에 직면  ->  심층신경망에 영향을 주며 하위층이 훈련하기 매우 어렵게 만듦.   
      
2. 대규모 신경망에서는 훈련이 극단적으로 느려질 것입니다.  
   
3. 수백만 개의 parameter를 가진 model은 훈련 set에 Overfitting될 위험이 매우 큼     

- Vanishing Gradient의 가장 널리 알려진 해결 방법으론 algorithm이 신경망의 모든 parameter에 대한 오차 함수의 gradient를 계산하면 경사하강법 단계에서 이 gradient를 사용하여 각 parameter를 수정함.    
   
   
- algorithm이 하위층으로 진행됨에 따라 gradient는 점점 작아지는 경우가 많습니다. 결국 경사 하강법이 하위층의 연결 가중치를 실제 변경되지 않은 챌 둔다면 훈련이 좋은 solution으로 수렴되지 않습니다.
             => 이 문제를  Vanishing Gradient   
      
       
- algorithm이 하위층으로 진행됨에 따라 gradient는 점점 커져 여러개의 층이 비정상적으로 큰 가중치로 갱신되면 algorithm은 발산하게 됨
            => 이 문제를  Exploding Gradient (주로 순환 신경망에서 나타남)   
   
- 일반적으로 심층 신경망은 불안정한 Gradient에 영향을 많이 받음. (여러 층이 서로 다른 속도로 학습될 수 있기 때문)

활성화 함수와 초기화 방식을 사용 했을 때 각 층에서 출력의 분산이 입력의 분산보다 더 크다는 것을 밝힘.   
-> 신경망의 위쪽으로 갈수록 층을 지날 때마다 분산이 계속 커져 가장 높은 층에서는 활성화 함수가 0이나 1로 수렴.
이는 로지스틱 함수의 평균이 0이아니라 0.5라는 사실 때문에 더 나빠집니다. ( hyperbolic tan 함수는 평균이 0이므로 깊은신경망에서 로지스틱 보다 조금 더 낫습니다.)

Logistic 함수는 항상 양수를 출력하므로 출력의 가중치 합이 입력보다 커질 가능성이 높음 => 이를 편향이동이라고 함

- tanh함수는 아래의 그림과 같이 입력값의 총합을 -1에서 1사이의 값으로 변환해 주며, 원점 중심(zero-centered)이기 때문에, 시그모이드와 달리 편향 이동이 일어나지 않는다. 하지만, tanh함수 또한 입력의 절대값이 클 경우 -1이나 1로 수렴하게 되므로 그래디언트를 소멸시켜 버리는 문제가 있다.


# 평균이 0인게 왜 0이 아닌 것보다 좋은 것일까?

Logistic 활성화 함수를 보면 입력의 절댓값이 크면 0이나 1로 수렴해서 기울기가 0에 매우 가까워지는 것을 알 수 있음.  
역전파가 될 때 사실상 신경망으로 전파시킬 gradient가 거의 없고 조금 있는 gradient는 최상위층에서부터 역전파가 진행됨에 따라 점차 약해져서 실제로 아래 층에는 아무것도 도달하지 않음.

## 세이비어 초기화와 He 초기화

글로럿과 벤지오는 새로운 방법을 제안   
- 예측 시에는 정방향, gradient를 역전파할 때는 역방향으로 양방향 신호가 적절하게 흐르도록.  
(조건 : 신호가 죽거나 폭주 또는 소멸하지 않아야함, 적절한 신호가 흐르기 위해서는 각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 함,역방향에서 층을 통과하기 전과 후의 gradient 분산이 동일해야 함. <논문 참고> )   
     
층의 입력과 출력 연결 개수가 같지 않다면 위 두 가지를 보장할 수가 없다.   
     
=> 이들의 실전에서 매우 잘 작동한다고 입증된 대안 : 연결 가중치를 356p 식 11-1에서 기술한 방식대로 무작위로 초기화하는 것.
이 식에서 $n_{inputs}$과 $n_{outputs}$는 가중치를 초기화하려는 층의 입력과 출력 연결의 개수임.(or fan-in과 fan-out으로 부름)      
     
=> 이 초기화 전력을 세이비어 초기화(Xavier initialization) or 글로럿 초기화(Glorot initialization)라고 함.   
    
Xavier Initialization (hyperbolic tan 활성화 함수를 사용 시)    
&nbsp;&nbsp; 평균이 0이고 표준편차 $\sigma = \sqrt{ \frac {2} {n_{inputs} + n_{outputs}}}$ 인 정규분포     
&nbsp;&nbsp; 또는 $ r = \sqrt{\frac {6} {n_{inputs} + n_{outputs}} }$일 때 -r과 +r 사이의 균등분포(uniform)    
      
=> 입력 연결개수와 출력 연결 개수가 비슷하면 $n_{inputs} == n_{outputs}$라고 하고 사용    
Xavier initialization 전략을 사용하며 훈련속도를 상당히 높일 수가 있었고 이것이 현재의 Deep Learning의 성공을 견인한 기술 중 하나임.   
     
다른 활성화 함수에 대해서는 ReLU 활성화 함수, ELU 활성화 함수(ReLU 변종) 등이 있다.  

### 활성화 함수 Logistic의   
&nbsp;&nbsp; 균등분포 [$-r, r$] &nbsp;&nbsp; $ r = 4\sqrt{\frac {6} {n_{inputs} + n_{outputs}}} $    
&nbsp;&nbsp; 정규분포 $\sigma = 4\sqrt{\frac {2} {n_{inputs} + n_{outputs}} } $

### 활성화 함수 Hyperbolic tan의   
&nbsp;&nbsp; 균등분포 [$-r, r$] &nbsp;&nbsp; $ r = \sqrt{\frac {6} {n_{inputs} + n_{outputs}}} $    
&nbsp;&nbsp; 정규분포 $\sigma = \sqrt{\frac {2} {n_{inputs} + n_{outputs}} } $

### 활성화 함수 ReLU와 그 변종들의   
&nbsp;&nbsp; 균등분포 [$-r, r$] &nbsp;&nbsp; $ r = \sqrt{2}\sqrt{\frac {6} {n_{inputs} + n_{outputs}}} $    
&nbsp;&nbsp; 정규분포 $\sigma = \sqrt{2}\sqrt{\frac {2} {n_{inputs} + n_{outputs}} } $

#### 활성화 함수를 잘못 선택하면  vanishing gradient 혹은 exploding gradient로 이어질 수가 있다.
#### 과거엔 sigmoid 활성화 함수가 최선의선택이라고 생각했지만 현재는 다른 활성화 함수(상황에 맞는)가 심층 신경망에서 훨씬 더 잘 작동한다고밝혀짐.

In [ ]:
he_init = tf.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.relu,
                         kernel_initializer = he_init, name = "hidden1")

## ReLU함수는 빠른 계산속도와 특정 양숫값에 수렴하지 않는다는 큰 장점이 있지만 (훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않는다는 의미로 죽었다고 말함) 죽은 ReLU라는 단점이 존재.

예시로 큰 학습률을 사용하면 신경망의 뉴런 절반이 죽어 있기도 함.   
훈련 도중 뉴런의 가중치가 바뀌어 가중치 합이 음수가 되면 그 다음부터 0을 출력하기 시작.   
ReLU 함수는 입력이 음수면 gradient가 0이 되기에 뉴런이 죽고 다시 살아나기 어렵.

이러한 문제 해결을 위하여 LeakyReLU와 같은 변종을 사용.    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; LeakyReLU$_{\alpha}$($z$) = max($\alpha, z$)로 정의   
Hyperparameter $\alpha$가 이 함수가 'leaky(새는)'정도를 결정.    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;새는 정도란 $z$ < 0 일 때 함수의 기울기, 일반적으로 0.01로 설정.     
-> 이 작은 기울기가 LeakyReLU를 절대 죽지 않게 만들어줌.   (뉴런이 혼수 상태에 오래 있을 수는 있찌만 다시 깨어날 가능성을 얻음)

### 최근 한 논문에서 여러 ReLU 함수의 변종을 비교해 얻은 결론으로 LeakyReLU가 ReLU보다 항상 성능이 높다.    
사실 $\alpha$ = 0.2(많이 통과)로 하는 것이 $\alpha$ = 0.01(조금 통과)보다 더 나은 성능을 내는 것을 보임

### 이 논문은 훈련하는 동안 주어진 범위에서 $\alpha$를 무작위로 선택하고 Test 시에는 평균을 사용하는 RReLU(Randomized leaky ReLU)도 평가함.

### 이 함수도 꽤 잘 작동했으며 (훈련 set의 overfitting 위험을  줄이는) 규제의 역할을 하는 것처럼 보임.

### 마지막을 $\alpha$가 훈련하는 동안 학습되는 PReLU(Parametric leaky ReLU) 즉, hyperparameter가 아니고 다른 model parameter와 마찬가지로 역전파에 의해 변경됨.    
이 함수는 대규모 Image data set에서는 ReLU보다 성능이 크게 앞서지만, 소규모 data set에서는 훈련 set에 overfitting 위험이 있음.

## ELU(Exponential Linear Unit)란 활성화 함수는 실험에서 다른 모든 ReLU 변종의 성능을 앞지름

훈련시간이 줄고 신경망의 test set 성능도 더 높았음. &nbsp;&nbsp;

   $ ELU_{\alpha}(z) = \alpha(exp(z) - 1),  z < 0$ 일 때 $/ z ,   z>=0$일 때

# ELU 특징   
1. $z < 0 $일 때 음숫값이 들어오므로 활성화 함수의 평균 출력이 0에 더 가까워짐.    
&nbsp;&nbsp;&nbsp;&nbsp; 이는 앞서 이야기한 vanishing gradient 완화.
2. hyperparameter $\alpha$는 $z$가 큰 음숫값일 때 ELU가 수렴할값을 정의함 , 보통 1로 설저아지만 필요시 변경 가능     

3. z<0이어도 gradient가 0이 아니므로 죽은 뉴런을 만들지 않음    

4. $\alpha = 1$일 때는 이 함수는 $z$ = 0에서 급격히 변동하지않고 $z = 0$을 포함해 모든구간에서 매끄러워 경사하강법의 속도를 올려줌      
5. 단점으로 ReLU나 기타 그 외에 변종들보다 계산이 느림.(지수함수를 사용하기 때문)

훈련하는 동안에는 수렴속도가 빠르기 때문에 상쇄돼지만 테스트 시에는 5.에 의해서 ELU 신경망이 ReLU 신경망보다 느림

## 심층 신경망의 은닉층에 활성화 함수는 일반적으로 ELU > LeakyReLU(그리고 변종들) > ReLU >tanh > Logistic 순서

## 실행속도가 중요하다면 LeakyReLU가 ELU보다 나음

### Hyperparameter를 더 추가하고 싶지않다면 기본 $\alpha$ 값을 사용(LeakyReLU는 0.01, ELU는 1).

### 시간과 computing power가 충분 시엔 교차 검증을 사용해 여러 활성화 함수를 평가해볼 수 있음.
### 신경망이 Overfitting되었다면 RReLU, 훈련 Set이 아주 크다면 PReLU를 포함시키면 좋음.

In [ ]:
hidden1 = tf.layers.dense(X, n_hidden1, activation = tf.nn.elu, name = "hidden1")

## Tensorflow에서 LeakyReLU를 기본으로 제공하지 않기에 만들어야 함.

In [ ]:
def leaky_relu(z, name = None) :
    return tf.maximum( 0.01 * z, z, name = name)

hidden1 = tf.layers.dense(X, n_hidden1, activation = leaky_relu, name = "hidden1")

## 11.1.3 Batch Normalization(BN, 배치 정규화)

ELU(또는 다른 ReLU 변종들)와 함께 He 초기화를 사용하면 훈련 초기 단계에서 Vanishing Gradient or Exploading Gradient 문제를 크게 감소시킬 수 있찌만 , 훈련하는 동안 재발생의 위험이 있음.

Sergey Ioffe(세르게이 아이오페), Christiam Szegedy(크리스티안 세게디)가 위 문제를 해결하기 위해 BN(Batch Normalization)기법을 제안.

일반적으로 훈련하는 동안 이전 층의 parameter가 변함에 따라 각 층에 들어오는 입력의 분포가 변화되는 문제!   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;=> 내부 공변량 변화 (Internal Covariate Shift)

이러하면 각 층에서 활성화 함수를 통과힉 전에 model에 연산을 하나 추가합니다.   
입력 데이터의 평균을 0으로 한 후 정규화한 다음 각 층에서 두 개의 새로운 parameter로 결괏값의 scale을 조정하고 이동시켜야함.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;=> model이 층마다 (1. scale 조정 2. 이동) 입력 데이터의 최적 scale과 평균을 학습.

### Batch Normalization Algorithm

$ \mu_{\beta} = \frac {1} {m_{B}} \sum_{i=1}^{m_{B}}{x^{(i)}}$  
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; => $\mu_{B}$는 미니배치 $B$에 대해 평가하여 관측한 평균입니다.    
         
$ \sigma_{B}^{2} = \frac {1} {m_{B}} \sum_{i=1}^{m_{B}} (x^{(i)} - \mu_{B})^{2} $     
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; => $\sigma_{B}$는 미니배치 $B$에 대해 평가하여 관측한 표준편차입니다.
    
$m_{B}$는 미니배치 $B$에 있는 샘플 수 입니다.
        
$ \hat{x}^{(i)} = \frac { x^{(i)} - \mu_{B} } {\sqrt{ \sigma_{B}^{2} + \epsilon }}$
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; => $\hat{x}^{(i)}$는 평균이 0이고 정규화된 입력입니다.
   
$ z^{(i)} = \gamma \hat{x}^{i} + \beta$
   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; => $\gamma$는 층의 scale된 parameter입니다. $\beta$는 층의 이동 (편향) parameter입니다.

### TIP
ReLU6 : 최댓값이 6인 ReLU6( ReLU6(z) = min(max(z, 0), 6) ) 
SELU : 신경망에 정규화를 주입하는 효과를 내는 함수       
Swish(z) = z sigmoid(z)로 비교적 간단하지만 다른 ReLU계열 함수보다 우수한 성능을 냈고 Tensorflow 1.5에 추가됨.   
     
       
        
공변량 변화(covariate shift)는 Test Data의 분포가 훈련 data와 다른 경우를 말함.
내부 공변량 변화(Internal Covariate Shift)는 신경망 층 사이에서 발생하는 입력 데이터의 분포 변화를 의미    
    
      
$ \gamma$와 $\beta$의 차원은 모두 $z^{(i)}$와 같음.   
층의 뉴런마다 $ \gamma$와 $\beta$를 하나씩 갖음     
11.3절에서  고속 Optimizer의 최적화 Algorithm에서 나오는 $\beta$는 hyperparameter며 Batch Normalization의 $\beta$와는 다름.    
     
$ \epsilon$은 분모가 0이 되는 것을 막기 위한 작은 숫자(전형적으로 $10^{-5}$). => 안전을 위한 항(smothing term)    
     
     
$z^{(i)}$은 BN 연산의 출력. => 입력의 scale을 조정하고 이동시킨 것.

test할 땐 평균과 표준편차를 계산할 mini batch가 없으니 전체 훈련 test의 평균과 표준편차를 사용

=> 이 값들은 훈련과정에서 이동 평균(moving average)을 사용하여 효육적으로 계산됨    
전체적으로 보면 Batch Normalization된 층마다 $ \gamma(scale), \beta($이동$) , \mu($평균$), \sigma($표준편차$) $ 4개의 parameter가 학습됨.

=> 이 기법을 논문에서 실험했던 모든 심층 신경망의 성능이 크게 향상되었음.   
tanh, logistic 활성화 함수 같이 수렴되는 활성화 함수를 사용하더라도 vanishing gradient 문제가 크게 감소됨.   
network가 가중치 초기화에 훨씬 덜 민감해짐.   훨씬 큰 learning rate를 사용할 수 있어 학습 속도를 크게 개선해줌.    
가장 뛰어난 image classification model에 적용하면 batch normalization가 14배나 적은 훈련 단계에서 같은 정확도를 달성하고 상당한 차이를 원래 model을 앞지름.

BN화된 model의 Ensemble을 사용해 Image Net 분류의 가장 뛰어난 수준에 도달함.

결론적으로 Batch Normalization은 규제와 같은 역할을 하여 다른 규제 기법의 필요성을 줄여줌.

BUT 첫 번째 은닉층에 BN을 적용하면 입력 데이터를 정규화할 필요가 없어지지만 BN은 model의 복잡도를 키움.  실행시간면에서 손해.   
층마다 추가되는 계산이 신경망의 예측이 느려지게함. 예측이 매우 빨라야하면 BN를 사용하기 전에 ELU + He 초기화만으로 얼마나 잘 수행이 되는지 확인해보는 것이 좋음.

Gradient Descent는 각 층마다 최적의 scale과 shift를 위한 parameter를 찾느라 훈련 초기에는 오히려 느려질 수 있지만 적절한 값을 찾으면 점점 빨라짐

In [ ]:
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")

#훈련하는 동안에는 True로 그 외에는 False로 설정
training = tf.placeholder_with_default(False, shape = (), name = "training")

hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1")
# batch_normalization()가
# (train 시) 현재 minibatch의 평균과 표준편차를 사용할지 
# (test 시) 전체 훈련set에 대한 평균과 표준편차를 사용할지를 지정하는 함수. 
bn1 = tf.layers.batch_normalization(hidden1, training = training, momentum = 0.9 )
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = "hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training = training, momentum = 0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = "outputs")

logits = tf.layers.batch_normalization(logits_before_bn, training = training, momentum = 0.9)

완전 연결 층과 배치 정규화 층이 번갈아 나옴. tf.layers.dense()를 사용해 완전 연결 층을 생성.   
배치정규화 층 이후에 활성화 함수를 적용ㅎ기에 완전 연결 층에는 어떠한 활성화 함수도 지정하지 않음.    
그리고 training, momentum 매개변수와 함께 tf.layers.batch_normalization() 함수를 사용해 배치 정규화층을 만듦.   
BN Algorithm은 exponential decay(지수 감소)를 사용해 이동 평균(moving average)을 계산.   
그래서 momentum 매개변수가 필요.   새로운 값 $v$가 주어지면 이동 평균 $\hat{v}$은 $\hat{v} <- \hat{v} \times momentum + v \times(1-momentum) $

적절한 momentum 값은 일반적으로 1에 가까움. 예를 들면 0.9, 0.99, 0.999    
data set이 크고 mini batch가 작을 경우 9를 더 넣어 1에 가깝게 함.

In [ ]:
from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization, training = training, momentum = 0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name = "hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)

hidden12 = tf.layers.dense(bn1_act, n_hidden2, name = "hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = "outputs")
logits = my_batch_norm_layer(logits_before_bn)

- 다른 연구자들은 활성화 함수 이전이 아니라  이후에 배치 정규화를 두어도 괜찮고 성능이 더 좋다고 주장.   
- functools.partial()은 사실 클래스로 그 사용법은 max_norm_regularizer(), '맥스-노름 규제'와 유사하지만 실제로는 새로운 함수 객체를 반환하는것이 아님. 이 클래스의 객체는 매개변수로 넘겨진 batch_normalization() 함수와 매개변수를 저장하고 있다가 다음 번  호출에서 전달된 매개변수와 저장된 매개변수를 합쳐 batch_normalization() 함수를 호출함.

예를 들어 10개의 층 모두에 같은 활성화 함수, 초기호, 규제 등을 사용한다면 이런 기교가 훨씬 읽기 쉬운 코드를 만들어줄 것.